In [27]:
#Util functions to parse results 
import os
def get_accuracy_from_file(path, prefices):
    result = [0 for _ in prefices]
    with open(path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            for i, prefix in enumerate(prefices):
                if line.startswith(prefix):
                    #Remove prefix from line 
                    line_ = line[len(prefix):]
                    result[i] = float(line_.split(" ")[1])
    return result

def get_accuracy_and_fn_sorted(directory, prefices):
    result = []
    for file in os.listdir(directory):
        if file.endswith(".txt"):
            accuracy = get_accuracy_from_file(os.path.join(directory, file), prefices)
            avg_accuracy = sum(accuracy) / len(accuracy)
            result.append((avg_accuracy, file))
    result.sort(key=lambda x: x[0], reverse=True)
    return result

def get_best_accuracy_and_fn(directory, prefices):
    result = get_accuracy_and_fn_sorted(directory, prefices)
    return result[0]

In [ ]:
!launch_scripts/baseline.sh "photo,cartoon,sketch"

"""Baseline results:
Evaluate: photo: Accuracy: 95.93 - Loss: 0.0010716533411048842
Evaluate: cartoon: Accuracy: 54.65 - Loss: 0.010657542146767772
Evaluate: sketch: Accuracy: 40.49 - Loss: 0.015146310804575324
"""

In [35]:
#Activation shaping experiments
import os

def run_experiment(category, configurations):
  for configuration in configurations:
    alpha = configuration["a"]
    layers = configuration["LAYERS"]
    if ("EXTENSION" in configuration):
      extension = configuration["EXTENSION"]
    else:
      extension = ""
    !launch_scripts/activation_shaping_experiments.sh {category} "{extension}" {alpha} {layers}

#Experiments with single layers - try to find best alpha
def explore_different_alpha():
  c = []
  for layer in [0, 1, 4]:
    c = c + [
      {"a": 0.5, "LAYERS": f"{layer}"},
      {"a": 0.8, "LAYERS": f"{layer}"},
      {"a": 0.2, "LAYERS": f"{layer}"}
    ]
  run_experiment("photo,cartoon,sketch", c)


#Experiments with single layers
def explore_single_layers(alpha):
  c = []
  for layer in [ 6, 8, 10, 11, 14, 15]:
    c = c + [
      {"a": alpha, "LAYERS": f"{layer}"},
    ]
  run_experiment("photo,cartoon,sketch", c)

def finetune_alpha_by_layer(alphas, layer):
  c = []
  for a in alphas:
    c = c + [
      {"a": a, "LAYERS": f"{layer}"},
    ]
  run_experiment("photo,cartoon,sketch", c)

def finetune_alpha_by_layer(alphas, layer):
  c = []
  for a in alphas:
    c = c + [
      {"a": a, "LAYERS": f"{layer}"},
    ]
  run_experiment("photo,cartoon,sketch", c)

#explore_different_alpha()
#explore_single_layers(0.8)
#finetune_alpha_by_layer([0.7,0.9,0.99], 1)


[]
Applied to 1 layers
 23%|█████████▉                                  | 7/31 [00:00<00:03,  7.18it/s]
Traceback (most recent call last):
  File "/home/marco/Activation-Shaping-AML/main.py", line 286, in <module>
    main()
  File "/home/marco/Activation-Shaping-AML/main.py", line 219, in main
    train(model, data)
  File "/home/marco/Activation-Shaping-AML/main.py", line 159, in train
    evaluate(model, data['test'], extra_str="TEST WITHOUT BINARIZATION ")
  File "/home/marco/.local/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/home/marco/Activation-Shaping-AML/main.py", line 31, in evaluate
    acc_meter.update(logits, y)
  File "/home/marco/.local/lib/python3.10/site-packages/torchmetrics/metric.py", line 446, in wrapped_func
    update(*args, **kwargs)
  File "/home/marco/.local/lib/python3.10/site-packages/torchmetrics/classification/stat_scores.py", line 315, in update
    _multiclass_stat_scor

In [ ]:
#Results for activation shaping exp
def print_(l):
  for i in l:
    print(i)
    
#Get results 
def get_results():
  print("Best result found")
  prefices = [
    f"TEST WITH BINARIZATIONEvaluate: {domain}: Accuracy:" for domain in ["photo", "cartoon", "sketch"]
  ]
  print("With binarization during evaluation")
  print_(get_accuracy_and_fn_sorted("record/activation_shaping_experiments/photo,cartoon,sketch", prefices))

  prefices = [
    f"TEST WITHOUT BINARIZATION Evaluate: {domain}: Accuracy:" for domain in ["photo", "cartoon", "sketch"]
  ]
  print("Without binarization during evaluation")
  print_(get_accuracy_and_fn_sorted("record/activation_shaping_experiments/photo,cartoon,sketch", prefices))

get_results()

"""Comments on the results:
>Best results are found with the activation shaping applied to the first layers, especially to the second one.
  =>Not sure but i think first layers are bigger
>Best results are found with Alpha = 0.9. Meaning that losing too much information is not good.
  In a way I think this is working kind of like a dropout layer. 
  With Alpha > 0.9 performances go down.
>"""

In [29]:
#DOMAIN ADAPTATION STANDARD
import os
#TODO try to run with and without binarization in test mode
def run_experiment(category, configurations):
  for configuration in configurations:
    K = configuration["K"]
    record_mode = configuration["RECORD_MODE"]
    layers = configuration["LAYERS"]
    if ("EXTENSION" in configuration):
      extension = configuration["EXTENSION"]
    else:
      extension = ""
    !launch_scripts/domain_adaptation.sh {category} "{extension}" {K} {record_mode} {layers}



#Experiments with single layers - try to find most promising ones
def explore_different_layers():
  c = []
  for layer in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]:
    c = c + [
      {"K": 0, "LAYERS": f"{layer}", "RECORD_MODE": "threshold"}
    ]
  run_experiment("sketch", c)

#Experiments with single layers - try to find most promising ones
def explore_different_layers_with_topK():
  c = [
    #Simulate case of previous experiment where alpha was 0.9
    {"K": 0.9, "LAYERS": "1", "RECORD_MODE": "topk"},
  ]
  #Try other promising layers with topK
  for k in [0.4, 0.5, 0.8]:
    c = c + [
      {"K": k, "LAYERS": "1", "RECORD_MODE": "topk"},
      {"K": k, "LAYERS": "3", "RECORD_MODE": "topk"},
      {"K": k, "LAYERS": "9", "RECORD_MODE": "topk"},
      {"K": k, "LAYERS": "4", "RECORD_MODE": "topk"},
      {"K": k, "LAYERS": "12", "RECORD_MODE": "topk"},
      {"K": k, "LAYERS": "14", "RECORD_MODE": "topk"},
      {"K": k, "LAYERS": "11", "RECORD_MODE": "topk"},
      {"K": k, "LAYERS": "10", "RECORD_MODE": "topk"},
    ]
  c = [
     {"K": 0.8, "LAYERS": "15", "RECORD_MODE": "topk"},
  ]
  run_experiment("sketch", c)

def finetune_k():
  c = [
    {"K": 0.9, "LAYERS": "7", "RECORD_MODE": "topk"},
    {"K": 0.9, "LAYERS": "9", "RECORD_MODE": "topk"},
    {"K": 0.9, "LAYERS": "3", "RECORD_MODE": "topk"},
    {"K": 0.9, "LAYERS": "10", "RECORD_MODE": "topk"},

    {"K": 0.9, "LAYERS": "1", "RECORD_MODE": "topk"},
    {"K": 0.9, "LAYERS": "4", "RECORD_MODE": "topk"},

    {"K": 0.9, "LAYERS": "1,3", "RECORD_MODE": "topk"},
    {"K": 0.9, "LAYERS": "1,7", "RECORD_MODE": "topk"},
    {"K": 0.9, "LAYERS": "3,10", "RECORD_MODE": "topk"},
    {"K": 0.9, "LAYERS": "7, 9", "RECORD_MODE": "topk"},
    {"K": 0.9, "LAYERS": "7, 10", "RECORD_MODE": "topk"},
    {"K": 0.9, "LAYERS": "7,9,10", "RECORD_MODE": "topk"},
  ]
  run_experiment("sketch", c)
#explore_different_layers()
#explore_different_layers_with_topK()
#finetune_k()
"""TODO
Se risultati migliori non sono ottenuti su layer 1, prova ad applicare random map su layer 1, magari aiuta ulteriormente
Inoltre registra qualche statistica su come cambiano le attivazioni dei layer con random M ed M registrata"""
c = [
  {"K": 0.9, "LAYERS": "3,7", "RECORD_MODE": "topk"},
  {"K": 0.9, "LAYERS": "7,8", "RECORD_MODE": "topk"},
  {"K": 0.9, "LAYERS": "7,11", "RECORD_MODE": "topk"},
  {"K": 0.5, "LAYERS": "7", "RECORD_MODE": "topk"},
]
run_experiment("sketch", c)


[]
100%|███████████████████████████████████████████| 31/31 [00:02<00:00, 14.44it/s]
[]
100%|███████████████████████████████████████████| 31/31 [00:02<00:00, 14.57it/s]
[]
100%|███████████████████████████████████████████| 31/31 [00:02<00:00, 14.67it/s]
[]
  0%|                                                    | 0/16 [00:00<?, ?it/s]launch_scripts/domain_adaptation.sh: line 13: 89352 Killed                  python3 main.py --experiment=domain_adaptation --experiment_name=domain_adaptation/${target_domain}/ --dataset_args="{'root': 'data/PACS', 'source_domain': 'art_painting', 'target_domain': '${target_domain}', 'K': ${K}, 'RECORD_MODE': '${RECORD_MODE}', 'LAYERS_LIST': '${LAYERS_LIST}' }" --batch_size=128 --num_workers=5 --grad_accum_steps=1 ${EXTENSION_PARAMS}


In [28]:
#Results for domain adaptation
def print_(l):
  for i in l:
    print(i)
    
#Get results 
def get_results():
  print("Best result found")
  prefices = [
    "TEST SIMPLE Evaluate: sketch: Accuracy:"
  ]
  print("Without binarization during evaluation")
  print_(get_accuracy_and_fn_sorted("record/domain_adaptation/sketch", prefices))

  prefices = [
    "TEST SIMPLE BINARIZED Evaluate: sketch: Accuracy:"
  ]
  print("With binarization during evaluation")
  print_(get_accuracy_and_fn_sorted("record/domain_adaptation/sketch", prefices))

get_results()


Best result found
Without binarization during evaluation
(54.09, 'L_7__K__0.9__RECORD_MODE__topk-log.txt')
(53.81, 'L_7__K__0.8__RECORD_MODE__topk-log.txt')
(53.45, 'random_M_L_7__K__0.8__RECORD_MODE__topk-log.txt')
(51.59, 'random_M_L_7__K__0.9__RECORD_MODE__topk-log.txt')
(45.13, 'L_7__K__0__RECORD_MODE__threshold-log.txt')
(42.22, 'L_1,7__K__0.9__RECORD_MODE__topk-log.txt')
(42.0, 'L_3__K__0.5__RECORD_MODE__topk-log.txt')
(41.94, 'L_3__K__0.4__RECORD_MODE__topk-log.txt')
(39.32, 'L_9__K__0.4__RECORD_MODE__topk-log.txt')
(39.27, 'L_10__K__0.4__RECORD_MODE__topk-log.txt')
(38.92, 'L_3__K__0.8__RECORD_MODE__topk-log.txt')
(38.33, 'L_1__K__0.4__RECORD_MODE__topk-log.txt')
(37.97, 'L_3__K__0.9__RECORD_MODE__topk-log.txt')
(37.34, 'L_1__K__0.5__RECORD_MODE__topk-log.txt')
(37.29, 'L_1__K__0.8__RECORD_MODE__topk-log.txt')
(37.26, 'L_1__K__0.9__RECORD_MODE__topk-log.txt')
(36.04, 'L_8__K__0__RECORD_MODE__threshold-log.txt')
(35.4, 'L_14__K__0__RECORD_MODE__threshold-log.txt')
(35.15, 'L_11_

In [32]:
#Domain adaptation: produce some statistics
#DOMAIN ADAPTATION STANDARD
import os
def run_experiment(category, configurations):
  for configuration in configurations:
    K = configuration["K"]
    record_mode = configuration["RECORD_MODE"]
    layers = configuration["LAYERS"]
    if ("EXTENSION" in configuration):
      extension = configuration["EXTENSION"]
    else:
      extension = ""
    !launch_scripts/domain_adaptation.sh {category} "{extension}" {K} {record_mode} {layers}

ext = "--print_stats=1 --layers_only_for_stats=0,1,2,3,4,5 --extra_str=FOR_STATS"
c = [
  {"K": 0.9, "LAYERS": "1,4,7,10,14,15", "RECORD_MODE": "topk", "EXTENSION": ext},
]
run_experiment("sketch", c)


[0, 1, 2, 3, 4, 5]
100%|███████████████████████████████████████████| 31/31 [00:03<00:00, 10.08it/s]


In [ ]:
#Domain generalization
import os
#TODO: experiment with pairs randomized each time!!

def run_experiment(category, configurations):
  for configuration in configurations:
    K = configuration["K"]
    record_mode = configuration["RECORD_MODE"]
    layers = configuration["LAYERS"]
    if ("EXTENSION" in configuration):
      extension = configuration["EXTENSION"]
    else:
      extension = ""
    !launch_scripts/domain_generalization.sh {category} "{extension}" {K} {record_mode} {layers}


configurations = [
   ]
for d in ["sketch"]:
  run_experiment(d, configurations)
